# test data

In [1]:
taxi_test = pd.read_csv("/Users/jisuim/Desktop/tp_taxi/jisu/test.csv")

In [2]:
from math import cos, asin, sqrt
def dist(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295     #Pi/180
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return 12742 * asin(sqrt(a))

In [3]:
for i in range(len(taxi_test)):
    lat1 = taxi_test['pickup_latitude'][i]
    lon1 = taxi_test['pickup_longitude'][i]
    lat2 = taxi_test['dropoff_latitude'][i]
    lon2 = taxi_test['dropoff_longitude'][i]
    taxi_test.at[i, 'distance'] = dist(lat1, lon1, lat2, lon2)


In [4]:
### 시간대 19~00(Night) / 01~07(Morning) / 08~18(Duty) 로 분할하기
Morning = ['01','02','03','04','05','06','07']
Night = ['19', '20', '21', '22', '23', '00']
for i in range(len(taxi_test)):
    k = taxi_test['pickup_datetime'][i][11:13]
    if k in Morning:
        taxi_test.at[i, "time_n"] = "Morning"
    elif k in Night:
        taxi_test.at[i, "time_n"] = "Night"
    else:
        taxi_test.at[i, "time_n"] = "Duty"
        
     

In [9]:
dummy = pd.get_dummies(taxi_test['time_n'],prefix='time_n')

# train model

In [5]:
taxi = pd.read_csv("/Users/jisuim/Desktop/tp_taxi/jisu/train.csv")

In [6]:
from math import cos, asin, sqrt
def dist(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295     #Pi/180
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return 12742 * asin(sqrt(a))

In [7]:
for i in range(len(taxi)):
    lat1 = taxi['pickup_latitude'][i]
    lon1 = taxi['pickup_longitude'][i]
    lat2 = taxi['dropoff_latitude'][i]
    lon2 = taxi['dropoff_longitude'][i]
    taxi.at[i, 'distance'] = dist(lat1, lon1, lat2, lon2)

In [8]:
### 시간대 19~00(Night) / 01~07(Morning) / 08~18(Duty) 로 분할하기
Morning = ['01','02','03','04','05','06','07']
Night = ['19', '20', '21', '22', '23', '00']
for i in range(len(taxi)):
    k = taxi['pickup_datetime'][i][11:13]
    if k in Morning:
        taxi.at[i, "time_n"] = "Morning"
    elif k in Night:
        taxi.at[i, "time_n"] = "Night"
    else:
        taxi.at[i, "time_n"] = "Duty"
        
     

In [13]:
taxi.loc[taxi.distance > 200] = np.nan ##200km 넘는 데이터 제거
taxi.loc[taxi.trip_duration > 30000] = np.nan ##40000초(약 11시간)가 넘는 데이터 제거
taxi.loc[taxi.passenger_count == 0] = np.NAN   ### passenger 수가 0인 데이터 제거
taxi.dropna(inplace=True)

In [14]:
dummy_time_n = pd.get_dummies(taxi['time_n'],prefix='time_n')
dfX = pd.concat([dummy_time_n, taxi['distance']], axis=1)

In [15]:
y = list(taxi['trip_duration'])
model = sm.OLS(y, dfX)
result = model.fit()

In [16]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.591
Model:                            OLS   Adj. R-squared:                  0.591
Method:                 Least Squares   F-statistic:                 7.015e+05
Date:                Mon, 05 Mar 2018   Prob (F-statistic):               0.00
Time:                        23:35:28   Log-Likelihood:            -1.0900e+07
No. Observations:             1456542   AIC:                         2.180e+07
Df Residuals:                 1456538   BIC:                         2.180e+07
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
time_n_Duty      482.2225      0.564    855.

# apply to test data

In [17]:
dfX2 = pd.concat([dummy, taxi_test['distance']], axis = 1)

In [18]:
dfX2.head(3)

,time_n_Duty,time_n_Morning,time_n_Night,distance
0,0,0,1,2.746426
1,0,0,1,2.759239
2,0,0,1,1.306155


In [19]:
taxi_test.head(3)

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,distance,time_n
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N,2.746426,Night
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N,2.759239,Night
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N,1.306155,Night


In [20]:
test_y = result.predict(dfX2)

In [21]:
test_y.head(3)

0    676.496910
1    678.163749
2    489.134256
dtype: float64

In [24]:
submission_test = pd.concat([taxi_test['id'], test_y], axis=1)

In [25]:
submission_test.head(3)

,id,0
0,id3004672,676.496910
1,id3505355,678.163749
2,id1217141,489.134256


In [26]:
submission_test.to_csv('submission.csv', sep=',', na_rep='NaN')